# Tide Analysis demonstration for Acapulco tide gauge (IOC)

to make this notebook running, you need: the following packages: 

xarray, pandas, geopandas, ruptures, searvey, utide, cartopy, shapely, matplotlib

See [README.md](https://github.com/tomsail/analysea) for more information
### imports

In [ ]:
import json
import matplotlib.pyplot as plt
import utide
from analysea.utils import average, nd_format, shift_for_maximum_correlation, json_format
import pandas as pd
import numpy as np

%matplotlib widget

In [ ]:
from searvey import ioc
ioc_stations = ioc.get_ioc_stations()

## load test case : Acapulco (Mex)

but you can also try with the other stations in the data folder !

In [ ]:
_ioc_station = "acap2" 
df = ioc.get_ioc_station_data(_ioc_station)
df0 = ioc.get_ioc_station_data(_ioc_station)

In [ ]:
ioc_stations = ioc.get_ioc_stations()
acapulco = ioc_stations.iloc[5]

In [ ]:
df0.index = df0['time']
df0 = df0.drop('time', axis=1)

# Medthod 1: Load JSON file containing de-tide information (not working great)

In [ ]:
acap_jsonf = '../tests/data/processed/acap2.json'
with open(acap_jsonf, 'r') as f:
    acap_json = json.load(f)

In [ ]:
h=utide.reconstruct(df['time'],nd_format(acap_json))

In [ ]:
df0['retide'] = h.h

### adjust the eventual time and vertical offsets

In [ ]:
df0.rad = average(df0.rad)

In [ ]:
shifted_x, shifted_y, time = shift_for_maximum_correlation(df0.rad.values,df0.retide.values,df0.index)
df1 = pd.DataFrame({'retide': shifted_y, 'raw': shifted_x,'surge': shifted_x - shifted_y},index=time)

In [ ]:
fig, ax = plt.subplots()
df1.plot(ax=ax)

## method 2: live detide and reconstruct

In [ ]:
opts = {'conf_int': 'linear',
        'constit' : 'auto',
        'method' : 'ols', # ols is faster and good for missing data (Ponchaut et al., 2001)
        'order_constit' : 'frequency',
        'Rayleigh_min' : 0.97,
        'lat': acapulco.lat,
        'verbose' : True,
} # careful if there is only one Nan parameter, the analysis crashes

In [ ]:
from analysea.tide import tide_analysis

In [ ]:
df0 = ioc.get_ioc_station_data(_ioc_station)
df0.index = df0['time']
df0 = df0.drop('time', axis=1)

In [ ]:
df0['tide'],df0['surge'],coef = tide_analysis(df0.rad,opts)

In [ ]:
df0.rad = average(df0.rad)

In [ ]:
h=utide.reconstruct(df['time'],coef)
df0['tide'] = h

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
df0.plot(ax=ax)

# compare the 2 methods

In [ ]:
# main constituents 
ASTRO_PLOT = [ "M2", "S2", "N2", "O1", "K2", "K1", "NU2", "Q1", "L2", 
              "P1", "2N2", "M4", "MS4", "MM", "MU2", "SSA", "LDA2", "MF", "MSM", "MN4"]

In [ ]:
coef = json_format(coef)
coef['name']

In [ ]:
index = []
for ast in ASTRO_PLOT : 
    try :
        index.extend([coef['name'].index(ast)])
    except ValueError as err: 
        index.append(float('nan'))
        print('[WARN]',err)
index

In [ ]:
index2 = []
acap_json = json_format(acap_json)
for ast in ASTRO_PLOT : 
    try :
        index2.extend([acap_json['name'].index(ast)])
    except ValueError as err: 
        index2.append(float('nan'))
        print('[WARN]',err)
index2

In [ ]:
amps_coef = np.zeros(len(ASTRO_PLOT))
for i in range(len(ASTRO_PLOT)):
    if ~np.isnan(index[i]):
        amps_coef[i] = coef['A'][i]
    else: 
        amps_coef[i]= np.nan
# 
amps_json = np.zeros(len(ASTRO_PLOT))
for i in range(len(ASTRO_PLOT)):
    if ~np.isnan(index2[i]):
        amps_json[i] = acap_json['A'][i]
    else: 
        amps_json[i]= np.nan

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
x = np.arange(0, len(ASTRO_PLOT))
ax.bar(x - 0.2 , amps_coef, width= 0.4, label='live de-tide')
ax.bar(x + 0.2 , amps_json, width= 0.4, label='from JSON')
ax.set_xticks(x, ASTRO_PLOT)
ax.legend()
